In [12]:
def exp(exp_num):
    exp_name = f'exp_{str(exp_num).zfill(4)}'
    !pipenv install -r requirements.txt
    !python exp/{exp_name}/main.py

In [15]:
exp(10)

Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Requirements file provided! Importing into Pipfile...
Installing dependencies from Pipfile.lock (4a5bc1)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
/Users/nakagawaayato/git/2048_RL/exp/exp_0010/main.py:16: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path='config', config_name='config')
/Users/nakagawaayato/.local/share/virtualenvs/2048_RL-3g713a6k/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no

In [1]:
import pickle

In [16]:
with open('/Users/nakagawaayato/git/2048_RL/outputs/exp_0010/best_model.pkl', 'rb') as f:
    dic = pickle.load(f)

In [22]:
import numpy as np
lut = np.array(dic['model'])

In [24]:
np.argmax(lut)

405000

In [25]:
class LUT:
    def __init__(self):
        self.lut = []
        '''
            0  1  2  3
            4  5  6  7
            8  9  10 11
            12 13 14 15
        '''

        self.tuples = [
            [0, 1, 4, 5], [1, 2, 5, 6], [2, 3, 6, 7],
            [4, 5, 8, 9], [5, 6, 9, 10], [6, 7, 10, 11],
            [8, 9, 12, 13], [9, 10, 13, 14], [10, 11, 14, 15],
            [0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15],
            [0, 4, 8, 12], [1, 5, 9, 13], [2, 6, 10, 14], [3, 7, 11, 15]
        ]
        # self.tuples_arr = np.array(self.tuples)
        self.reset_model()

    def _state_convert(self, state):
        '''
            0, 2, 4, 8... -> 0, 1, 2, 3... に変換
        '''
        return np.log2(np.clip(state, a_min=1, a_max=None)).astype(np.int8)
    
    def get_value(self, state):

        '''
            state -> 
            for ntuple in ntuples:
                ntupleでindexに変換 -> 検索
        '''
        value = 0
        state = self._state_convert(state)
        for tuple_idx, tuple in enumerate(self.tuples):
            index = self._state2index(tuple, tuple_idx, state)
            value += self.lut[index]
        return value

    def _state2index(self, tuple, tuple_idx, state):
        idx_start = self.lut_split[tuple_idx]
        idx = sum(state.reshape(-1)[tuple] * (15 ** np.array([0, 1, 2, 3])))    
        return idx_start + idx

    def learn(self, state, grad):
        state = self._state_convert(state)
        for tuple_idx, tuple in enumerate(self.tuples):
            index = self._state2index(tuple, tuple_idx, state)
            self.lut[index] += grad
    
    def reset_model(self):
        lut_split = [0]
        length = 0
        for ntuple in self.tuples:
            length += 15**len(ntuple)
            lut_split.append(length)
        self.lut = [0.0] * length
        self.lut_split = lut_split
    
    def index2tuple(self, index):
        for i, split in enumerate(self.lut_split):
            if split <= index:
                tuple_idx = i
                idx = index - split
        # idx.
        # lut_split = [0]
        # length = 0
        # for ntuple in self.tuples:
        #     length += 15**len(ntuple)
        #     lut_split.append(length)

In [27]:
lut = LUT()

In [28]:
lut.lu